In [1]:
import os
import cv2
import numpy as np

def load_ur_fall_dataset(data_dir):
    images_dir = os.path.join(data_dir, 'images')
    labels_dir = os.path.join(data_dir, 'labels')

    images = []
    labels = []

    for img_file in os.listdir(images_dir):
        if img_file.endswith('.jpg') or img_file.endswith('.png'):
            img_path = os.path.join(images_dir, img_file)
            label_path = os.path.join(labels_dir, img_file.replace('.jpg', '.txt').replace('.png', '.txt'))

            # Baca gambar
            image = cv2.imread(img_path)
            images.append(image)

            # Baca label
            with open(label_path, 'r') as f:
                lines = f.readlines()
                label = [list(map(float, line.strip().split())) for line in lines]
                labels.append(label)

    return images, labels

In [2]:
import torch
from models.experimental import attempt_load
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint

# Load model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weights = torch.load('yolov7-w6-pose.pt', map_location=device)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

# Load dataset UR Fall
data_dir = 'test'  # Ganti dengan path ke folder dataset UR Fall
images, labels = load_ur_fall_dataset(data_dir)

# Inisialisasi metrik
true_positives = 0
false_positives = 0
false_negatives = 0

# Evaluasi pada setiap gambar
for img, label in zip(images, labels):
    # Preprocessing gambar
    img_resized = letterbox(img, 960, stride=64, auto=True)[0]
    img_tensor = transforms.ToTensor()(img_resized)
    img_tensor = torch.tensor(np.array([img_tensor.numpy()]))

    if torch.cuda.is_available():
        img_tensor = img_tensor.half().to(device)

    # Inferensi
    with torch.no_grad():
        output, _ = model(img_tensor)
    output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
    output = output_to_keypoint(output)

    # Proses hasil deteksi
    detected_fall = False
    for det in output:
        if len(det):
            detected_fall = True  # Asumsi deteksi jatuh jika ada keypoints terdeteksi

    # Bandingkan dengan ground truth
    ground_truth_fall = any(cls[0] == 0 for cls in label)  # Class 0: fall

    if detected_fall and ground_truth_fall:
        true_positives += 1
    elif detected_fall and not ground_truth_fall:
        false_positives += 1
    elif not detected_fall and ground_truth_fall:
        false_negatives += 1

# Hitung metrik
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (true_positives) / len(images)  # Akurasi sederhana

print(f"True Positives: {true_positives}")
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")
print(f"Accuracy: {accuracy:.4f}")

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5160\918565691.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load('yolov7-w6-pose.pt', map_location=device

NameError: name 'letterbox' is not defined